In [2]:
from pinecone import Pinecone
import pickle
from rag.pinecone_query import PineconeRetriever
from rag.pipeline import RAG_Piepline
from config import PINECONE_API_KEY, GROQ_API_KEY, LANGCHAIN_API_KEY

pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "textbook-rag"
mapping_path = "data/chunks/mapping.pickle"
embedding_mdl = "multilingual-e5-large"
k = 3

with open(mapping_path, 'rb') as f:
    mapping = pickle.load(f)

retriever = PineconeRetriever(pc=pc, index_name=index_name, embedding_mdl=embedding_mdl, k=k, mapping=mapping)
rag = RAG_Piepline(retriever, GROQ_API_KEY, LANGCHAIN_API_KEY)

In [3]:
reply = rag.get_response("List some uses of ultrasound")
print(reply)

Some uses of ultrasound are:

1. Cleaning parts located in hard-to-reach places.
2. Detecting cracks and flaws in metal blocks.
3. Echocardiography (forming the image of the heart).
4. Getting images of internal organs of the human body.
5. Detecting abnormalities, such as stones or tumours in different organs.


agent

In [5]:
from agent.agent import Agent
from rag.pinecone_query import PineconeRetriever
from pinecone import Pinecone
import pickle
from config import PINECONE_API_KEY, GROQ_API_KEY, LANGCHAIN_API_KEY

pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "textbook-rag"
mapping_path = "data/chunks/mapping.pickle"
embedding_mdl = "multilingual-e5-large"
k = 3

with open(mapping_path, 'rb') as f:
    mapping = pickle.load(f)

retriever = PineconeRetriever(pc=pc, index_name=index_name, embedding_mdl=embedding_mdl, k=k, mapping=mapping)
agent = Agent(GROQ_API_KEY, LANGCHAIN_API_KEY, retriever, model="mixtral-8x7b-32768")
reply = agent.query("Whats the color of banana")
print(reply)

0
[]
The color of a banana can vary depending on its stage of ripeness. A green banana is unripe and its color will be green. As it ripens, the banana turns yellow. A ripe banana is typically yellow, although some varieties may have spots of brown or black. Overripe bananas can become very dark brown or black. So, the common color of a banana is yellow.


img stuff

In [4]:
from config import PINECONE_API_KEY, GROQ_API_KEY, LANGCHAIN_API_KEY
from database_creation.image_data import get_image_caption

resp = get_image_caption("data/img_data/img8.png", "Low pitch sound has low frequency and high pitch of sound has high frequency", GROQ_API_KEY)
print(resp)

Based on the image, here is a 4-line description for the given low and high pitched sounds:

The image graphically illustrates the relationship between sound pitch and its corresponding wave shapes. A low pitched sound is depicted as a wave with a low amplitude and a long wavelength, resulting from low frequency oscillations. In contrast, a high pitched sound is represented by a wave with high amplitude and short wavelength due to its high frequency oscillations in the waveform.


In [1]:
from config import PINECONE_API_KEY, GROQ_API_KEY, LANGCHAIN_API_KEY
from database_creation.image_data import upsert_images

image_list_path = "data/img_data/index.txt"
image_path = "data/img_data/imgs/"

upsert_images(None, None, image_list_path, image_path, GROQ_API_KEY)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
['Vibrating tuning fork just touching the suspended table tennis ball. A vibrant, luminous orange dot is emanating from the tip of a vibrating tuning fork positioned just above a suspended table tennis ball.', 'One of the prongs of the vibrating tuning fork touching the water surface. In this image, a vibrating tuning fork with one prong touching the surface of a glass of water, causes circular waves on the water surface.', "Both the prongs of the vibrating tuning fork dipped in water. The image shows a tuning fork submerged in water, with movement in both prongs. The water is contained in a glass cup. The fork's prongs are visible and appear to be vibrating, creating a ripple effect in the water.", 'A vibrating object creating a series of compressions (C) and rarefactions (R) in the medium. A series of compressions (C) and rarefactions (R) in a medium, caused by a vibrating object, illustrating the formation of sound waves.', 'Longit